# ERS NIRCam Demo with Eureka!
## By the ERS NIRCam Team (Ahrer et al. 2022)
### Hello! This notebook shows a full demonstration reducing and analyzing the NIRCam/F322W2 transit of WASP-39b from the JWST Transiting Exoplanet Early Release Science Program (ERS 1366). Following this pipeline, you should be able to reproduce the NIRCam spectrum presented in the main text of Ahrer et al. (2022). 
### This data reduction uses the [Eureka!](https://github.com/kevin218/Eureka) pipeline. Before running this Jupyter Notebook, please follow the steps in "README.md" to set up an environment for this demonstration and install Eureka. Note that this demonstration is using a fixed version of Eureka! to ensure that no future updates break the example here, so even if you have the current version of Eureka! installed on your own machine you'll need to follow the set-up steps to ensure this demo works properly. For more information on Eureka! see its documentation or refer to Bell et al. (2022).

## First, download the data set.
### Data can be downloaded from MAST at: (DOI? Specify which file extensions/stage?)
### After downloading the data, place it in a directory "./Uncalibrated" within this tutorial.

## Next, let's import packages!

In [1]:
import sys
sys.path.append('./')
sys.path.insert(0,'./')
import eureka.S1_detector_processing.s1_process as s1
import eureka.S2_calibrations.s2_calibrate as s2
import eureka.S3_data_reduction.s3_reduce as s3
import eureka.S4_generate_lightcurves.s4_genLC as s4
import eureka.S5_lightcurve_fitting.s5_fit as s5
import eureka.S6_planet_spectra.s6_spectra as s6

eventlabel = 'nircam_wasp39b'
ecf_path = './ecf/'

## Eureka! is divided into six "Stages". Each Stage has a corresponding Eureka Control File, or .ecf file. These files are named "S2_eventlabel.ecf" where the first half of the name (S2, S3, etc.) refers to the Stage that .ecf file interacts with, and the second half refers to the "eventlabel" keyword we defined above. See the Eureka! documentation for a full description of each stage and all keywords in the .ecf files. Here we'll give a very brief summary of each Stage and discuss a few important keywords for reproducing the paper results.

## Stage 1: Correcting detector-level effects and fitting the up-the-ramp slope.
### Most important keywords in the .ecf:
#### 1. jump_rejection_threshold - this sets the sigma threshold for rejecting a jump in the up-the-ramp slope as due to a cosmic ray hit. The standard value in the jwst pipeline is 4.0, but this data reduction found a value of 6.0 produced better results.
#### 2. topdir - Edit this to make it the path to where you've downloaded the data for this demo. The path should look like this: /path/to/NIRCam_demo/wasp39b_data/
#### 3. inputdir - This keyword tells Eureka! where to look for the Uncalibrated outputs to feed into the Stage 1 code. This path is relative to "topdir". In this demo, we set inputdir=/Uncalibrated
#### 4. outputdir - This is where Eureka! will save all the Stage 2 outputs, including plots and log files. Here we set outputdir=/Stage1, which means the output files will be saved within a folder on the path /path/to/NIRCam_demo/NIRCam_full_data/Stage1/
#### Note that topdir, inputdir, and outputdir work the same in every .ecf file, so in each file edit them so that "inputdir" points to the previous Stage's outputs, and "outputdir" points to where you want to save that Stage's outputs.
### And that's it! Everything else in this step was left set to default values.

In [2]:
s1_meta = s1.rampfitJWST(eventlabel, ecf_path=ecf_path)


Starting Stage 1 Processing
Copying S1 control file

Found 4 data file(s) ending in uncal.fits
Starting file 1 of 4: jw01366002001_04103_00001-seg001_nrcalong_uncal.fits


2022-09-28 08:49:51,588 - stpipe.EurekaS1Pipeline - INFO - EurekaS1Pipeline instance created.
2022-09-28 08:49:51,590 - stpipe.EurekaS1Pipeline.group_scale - INFO - GroupScaleStep instance created.
2022-09-28 08:49:51,592 - stpipe.EurekaS1Pipeline.dq_init - INFO - DQInitStep instance created.
2022-09-28 08:49:51,594 - stpipe.EurekaS1Pipeline.saturation - INFO - SaturationStep instance created.
2022-09-28 08:49:51,596 - stpipe.EurekaS1Pipeline.ipc - INFO - IPCStep instance created.
2022-09-28 08:49:51,598 - stpipe.EurekaS1Pipeline.superbias - INFO - SuperBiasStep instance created.
2022-09-28 08:49:51,600 - stpipe.EurekaS1Pipeline.refpix - INFO - RefPixStep instance created.
2022-09-28 08:49:51,602 - stpipe.EurekaS1Pipeline.rscd - INFO - RscdStep instance created.
2022-09-28 08:49:51,604 - stpipe.EurekaS1Pipeline.firstframe - INFO - FirstFrameStep instance created.
2022-09-28 08:49:51,606 - stpipe.EurekaS1Pipeline.lastframe - INFO - LastFrameStep instance created.
2022-09-28 08:49:51,608

2022-09-28 08:49:53,820 - stpipe.EurekaS1Pipeline - INFO - Prefetching reference files for dataset: 'jw01366002001_04103_00001-seg001_nrcalong_uncal.fits' reftypes = ['dark', 'gain', 'linearity', 'mask', 'readnoise', 'refpix', 'reset', 'rscd', 'saturation', 'superbias']
2022-09-28 08:49:55,104 - stpipe.EurekaS1Pipeline - INFO - Prefetch for DARK reference file is '/Users/megan/crds_cache/references/jwst/nircam/jwst_nircam_dark_0355.fits'.
2022-09-28 08:49:55,105 - stpipe.EurekaS1Pipeline - INFO - Prefetch for GAIN reference file is '/Users/megan/crds_cache/references/jwst/nircam/jwst_nircam_gain_0097.fits'.
2022-09-28 08:49:55,105 - stpipe.EurekaS1Pipeline - INFO - Prefetch for LINEARITY reference file is '/Users/megan/crds_cache/references/jwst/nircam/jwst_nircam_linearity_0052.fits'.
2022-09-28 08:49:55,105 - stpipe.EurekaS1Pipeline - INFO - Prefetch for MASK reference file is '/Users/megan/crds_cache/references/jwst/nircam/jwst_nircam_mask_0063.fits'.
2022-09-28 08:49:55,106 - stpip

2022-09-28 08:53:01,280 - stpipe.EurekaS1Pipeline.linearity - INFO - Using Linearity reference file /Users/megan/crds_cache/references/jwst/nircam/jwst_nircam_linearity_0052.fits
2022-09-28 08:53:01,430 - stpipe.EurekaS1Pipeline.linearity - WARNING - Keyword UAZ_DO_NOT_USE does not correspond to an existing DQ mnemonic, so will be ignored
2022-09-28 08:53:01,431 - stpipe.EurekaS1Pipeline.linearity - WARNING - Keyword UAZ_NO_LIN_SAMP does not correspond to an existing DQ mnemonic, so will be ignored
2022-09-28 08:53:01,431 - stpipe.EurekaS1Pipeline.linearity - WARNING - Keyword UAZ_BAD_LIN_FIT does not correspond to an existing DQ mnemonic, so will be ignored
2022-09-28 08:53:01,431 - stpipe.EurekaS1Pipeline.linearity - WARNING - Keyword UAZ_NO_WELL_SAMP does not correspond to an existing DQ mnemonic, so will be ignored
2022-09-28 08:53:01,432 - stpipe.EurekaS1Pipeline.linearity - WARNING - Keyword UAZ_MODEL_FIT_FAIL does not correspond to an existing DQ mnemonic, so will be ignored
202

2022-09-28 08:54:15,495 - stpipe.EurekaS1Pipeline.jump - INFO - From highest outlier Two-point found 0 pixels with at least one CR and three groups
2022-09-28 08:54:15,495 - stpipe.EurekaS1Pipeline.jump - INFO - From highest outlier Two-point found 640 pixels with at least one CR and two groups
2022-09-28 08:54:16,602 - stpipe.EurekaS1Pipeline.jump - INFO - Working on integration 7:
2022-09-28 08:54:16,793 - stpipe.EurekaS1Pipeline.jump - INFO - From highest outlier Two-point found 35499 pixels with at least one CR and at least four groups
2022-09-28 08:54:16,794 - stpipe.EurekaS1Pipeline.jump - INFO - From highest outlier Two-point found 0 pixels with at least one CR and three groups
2022-09-28 08:54:16,794 - stpipe.EurekaS1Pipeline.jump - INFO - From highest outlier Two-point found 647 pixels with at least one CR and two groups
2022-09-28 08:54:17,888 - stpipe.EurekaS1Pipeline.jump - INFO - Working on integration 8:
2022-09-28 08:54:18,081 - stpipe.EurekaS1Pipeline.jump - INFO - From

...

/Users/megan/opt/anaconda3/envs/eureka-test/lib/python3.9/site-packages/stcal/ramp_fitting/ols_fit.py:1155: RuntimeWarning: invalid value encountered in multiply
  var_p4[num_int, :, :, :] *= (segs_4[num_int, :, :, :] > 0)
/Users/megan/opt/anaconda3/envs/eureka-test/lib/python3.9/site-packages/stcal/ramp_fitting/ols_fit.py:1155: RuntimeWarning: invalid value encountered in multiply
  var_p4[num_int, :, :, :] *= (segs_4[num_int, :, :, :] > 0)
/Users/megan/opt/anaconda3/envs/eureka-test/lib/python3.9/site-packages/stcal/ramp_fitting/ols_fit.py:1155: RuntimeWarning: invalid value encountered in multiply
  var_p4[num_int, :, :, :] *= (segs_4[num_int, :, :, :] > 0)
/Users/megan/opt/anaconda3/envs/eureka-test/lib/python3.9/site-packages/stcal/ramp_fitting/ols_fit.py:1155: RuntimeWarning: invalid value encountered in multiply
  var_p4[num_int, :, :, :] *= (segs_4[num_int, :, :, :] > 0)
/Users/megan/opt/anaconda3/envs/eureka-test/lib/python3.9/site-packages/stcal/ramp_fitting/ols_fit.py:1155: 

/Users/megan/opt/anaconda3/envs/eureka-test/lib/python3.9/site-packages/stcal/ramp_fitting/ols_fit.py:1155: RuntimeWarning: invalid value encountered in multiply
  var_p4[num_int, :, :, :] *= (segs_4[num_int, :, :, :] > 0)
/Users/megan/opt/anaconda3/envs/eureka-test/lib/python3.9/site-packages/stcal/ramp_fitting/ols_fit.py:1155: RuntimeWarning: invalid value encountered in multiply
  var_p4[num_int, :, :, :] *= (segs_4[num_int, :, :, :] > 0)
/Users/megan/opt/anaconda3/envs/eureka-test/lib/python3.9/site-packages/stcal/ramp_fitting/ols_fit.py:1155: RuntimeWarning: invalid value encountered in multiply
  var_p4[num_int, :, :, :] *= (segs_4[num_int, :, :, :] > 0)
/Users/megan/opt/anaconda3/envs/eureka-test/lib/python3.9/site-packages/stcal/ramp_fitting/ols_fit.py:1155: RuntimeWarning: invalid value encountered in multiply
  var_p4[num_int, :, :, :] *= (segs_4[num_int, :, :, :] > 0)
/Users/megan/opt/anaconda3/envs/eureka-test/lib/python3.9/site-packages/stcal/ramp_fitting/ols_fit.py:1155: 

/Users/megan/opt/anaconda3/envs/eureka-test/lib/python3.9/site-packages/stcal/ramp_fitting/ols_fit.py:1155: RuntimeWarning: invalid value encountered in multiply
  var_p4[num_int, :, :, :] *= (segs_4[num_int, :, :, :] > 0)
/Users/megan/opt/anaconda3/envs/eureka-test/lib/python3.9/site-packages/stcal/ramp_fitting/ols_fit.py:1155: RuntimeWarning: invalid value encountered in multiply
  var_p4[num_int, :, :, :] *= (segs_4[num_int, :, :, :] > 0)
/Users/megan/opt/anaconda3/envs/eureka-test/lib/python3.9/site-packages/stcal/ramp_fitting/ols_fit.py:1155: RuntimeWarning: invalid value encountered in multiply
  var_p4[num_int, :, :, :] *= (segs_4[num_int, :, :, :] > 0)
/Users/megan/opt/anaconda3/envs/eureka-test/lib/python3.9/site-packages/stcal/ramp_fitting/ols_fit.py:1155: RuntimeWarning: invalid value encountered in multiply
  var_p4[num_int, :, :, :] *= (segs_4[num_int, :, :, :] > 0)
/Users/megan/opt/anaconda3/envs/eureka-test/lib/python3.9/site-packages/stcal/ramp_fitting/ols_fit.py:1155: 

2022-09-28 09:03:01,881 - stpipe.EurekaS1Pipeline.gain_scale - INFO - Step gain_scale done
2022-09-28 09:03:01,954 - stpipe.EurekaS1Pipeline.gain_scale - INFO - Step gain_scale running with args (<CubeModel(119, 256, 2048) from jw01366002001_04103_00001-seg001_nrcalong_uncal.fits>,).
2022-09-28 09:03:01,955 - stpipe.EurekaS1Pipeline.gain_scale - INFO - Step gain_scale parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': 'gain_scaleints', 'search_output_file': True, 'input_dir': '/Users/megan/Documents/Code/ERS_NIRCam/ERS_NIRCam/Uncalibrated'}
2022-09-28 09:03:01,992 - stpipe.EurekaS1Pipeline.gain_scale - INFO - GAINFACT not found in gain reference file
2022-09-28 09:03:01,993 - stpipe.EurekaS1Pipeline.gain_scale - INFO - Step will be skipped
2022-09-28 09:03:01,996 - stpipe.EurekaS1Pipeline.gain_scale - INFO - Step gain_scal

ValidationError: 'CALCULATED_TRACK_TR_202111' is not one of ['CALCULATED', 'CALCULATED_COARSE_TR_202107', 'CALCULATED_FULL', 'CALCULATED_FULLVA', 'CALCULATED_GSCMD', 'CALCULATED_GSCMD_V3PAGS', 'CALCULATED_ORIGINAL', 'PLANNED', 'CALCULATED_TR_202105', 'CALCULATED_TR_202105_VA', 'CALCULATED_TRACK_TR_202107']

Failed validating 'enum' in schema['properties']['meta']['properties']['visit']['properties']['engdb_pointing_quality']:
    OrderedDict([('title', 'Quality of pointing information from EngDB'),
                 ('type', 'string'),
                 ('enum',
                  ['CALCULATED',
                   'CALCULATED_COARSE_TR_202107',
                   'CALCULATED_FULL',
                   'CALCULATED_FULLVA',
                   'CALCULATED_GSCMD',
                   'CALCULATED_GSCMD_V3PAGS',
                   'CALCULATED_ORIGINAL',
                   'PLANNED',
                   'CALCULATED_TR_202105',
                   'CALCULATED_TR_202105_VA',
                   'CALCUL

## Stage 2: Additional pre-spectral-extraction steps, such as assignment of the world coordinate system, flat fielding, wavelength calibration. Outputs calibrated 2D images.
### Most important keywords in the .ecf:
#### 1. skip_bkg_subtract - Kevin had this set to false?? Check this.


In [ ]:
s2_meta = s2.calibrateJWST(eventlabel, ecf_path=ecf_path, s1_meta=s1_meta)

## Stage 3: Identify source position, perform background subtraction, perform spectral extraction to produce time series of 1D extracted spectra.
### Most important keywords in the .ecf:
#### 1. ncpu - Number of CPUs on your machine. This won't affect the data reduction except to make it run faster if you are able to run it on more CPUs. You can change this number to fit whatever machine you're running on.
#### 2. ywindow and xwindow - These specify the region of the image that you're interested in performing background subtraction and spectral extraction on. Note that this is the *full* image, not just the window surrounding the spectral trace. For this demo, the ywindow is set to ignore reference pixels on the edges of the detector, and the xwindow is set to select the region of higher throughput where the spectral trace can actually be seen in the image. These values were selected based on viewing an image in ds9 or another fits file viewer and looking at the position of the spectral trace.
### The next several parameters have to do with the background subtraction. Eureka! performs background subtraction by identifying the spectral trace, masking out a region surrounding the spectral trace, and using the remaining pixels as the background.
#### 3. bg_hw - Defines the half-width of the masked area not included in background subtraction. In this example, a value of 14 means that 14 pixels both above and below the identified source position are excluded from each column. We tested a few different values and found that 14 minimized the MAD of the resulting light curves, as it was large enough to exclude contamination from the spectrum but not so large that the background subtraction was affected by having less pixels to estimate the background from.
#### 4. bg_deg - The background is subtracted by doing a column-by-column polynomial fit, and bg_deg defines the degree of that fit. Setting bg_deg = -1 will just calculate and subtract out a median for each column. For this reduction, bg_deg=1 removed the background sufficiently well.
### Now some parameters for how we'll do the spectral extraction!
#### 5. spec_hw - Defines the half-width of the region you extract the spectrum from. In this example, a value of 9 means that the extraction will be perfomed on a box of pixels extending 9 up and 9 down from the identified source position. We tested several values and found that a half-width of 9 minimized the MAD of the resulting light curves, as it was wide enough to catch the edges of the spectrum but not so wide that it added too much background contamination.
### Finally, some parameters for printing diagnostics and saving output.
#### 6. isplots_S3 - How many plots do you want to create? This can be set to 1, 3, or 5, where a bigger number will print out more different types of diagnostic plots. The default in this demo is 5 so that you can see all the diagnostics.

## Stage 4: Convert 1D extracted spectra to time series light curves

## Stage 5: Light Curve Fitting

## Stage 6: Create Final Spectrum